# Explore Results

In [17]:
import sys

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from pathlib import Path
import pandas as pd
import os
import joblib
import pickle
import plotly.graph_objects as go
import plotly.express as px
from src.sklearn_utils import predict_fn, plot_predict_vs_ground_truth, get_naive_baselines

In [19]:
artifacts_dir = Path(os.path.abspath('')).parent / 'artifacts'
model_path = artifacts_dir / 'gradient_boosting-20220704-141541.joblib'
model = joblib.load(model_path)

In [24]:
model.cv_results_

{'mean_fit_time': array([ 22.77074528,  66.01753516, 130.1483892 ]),
 'std_fit_time': array([ 0.6903504 ,  1.79167142, 46.180696  ]),
 'mean_score_time': array([0.04763288, 0.0902936 , 0.05909877]),
 'std_score_time': array([0.00286024, 0.08205299, 0.02113134]),
 'param_clf__base_estimator__max_depth': masked_array(data=[2, 4, 6],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__base_estimator__max_depth': 2},
  {'clf__base_estimator__max_depth': 4},
  {'clf__base_estimator__max_depth': 6}],
 'split0_test_score': array([0.07306397, 0.07205387, 0.07474747]),
 'split1_test_score': array([0.05938567, 0.05699659, 0.05836177]),
 'split2_test_score': array([0.06622517, 0.06390728, 0.06523179]),
 'split3_test_score': array([0.05785953, 0.05785953, 0.05719064]),
 'split4_test_score': array([0.0620339 , 0.06372881, 0.06237288]),
 'mean_test_score': array([0.06371365, 0.06290922, 0.06358091]),
 'std_test_score': array([0.00546919, 0

In [26]:
mlb = pickle.load(open(artifacts_dir / 'gradient_boosting-20220704-141541-binarizer.pkl', 'rb'))

In [53]:
df = pd.read_csv(artifacts_dir / 'gradient_boosting-20220704-141541-test.csv')
# convert strings in column to list. e.g. "['A', 'B', 'C']" -> ['A', 'B', 'C'], "['Hello', 'World']" -> ['Hello', 'World']
df['sectors_list'] = df['sectors_list'].apply(lambda x: eval(x))
df["sectors_list"] = df["sectors_list"].apply(
    lambda x: [tag for tag in x if tag in mlb.classes_.tolist()]
)

In [ ]:
y_true, y_pred, hl = predict_fn(df, mlb, model)

In [ ]:
naive_baselines = get_naive_baselines(df, mlb)

In [60]:
fig = plot_predict_vs_ground_truth(y_true, y_pred, mlb)
fig.show()